<a href="https://colab.research.google.com/github/codeskavy/VirtualTryon/blob/main/Vton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

MOUNTING THE GOOGLE DRIVE

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
drive_path = "/content/drive/My Drive"

INSTALL LIBRARIES

In [3]:
!pip3 install numpy

IMPORT

In [4]:
import os
import numpy as np
import cv2
from glob import glob
import tensorflow as tf
from tensorflow.keras.layers import Conv2D,BatchNormalization,Activation,MaxPool2D,Conv2DTranspose,Concatenate,Input
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau,CSVLogger


SEEDING

In [5]:
os.environ["PYTHONHASHSEED"]=str(42)
np.random.seed(42)
tf.random.set_seed(42)


HYPERPARAMETERS

In [14]:
batch_size=4
lr=1e-4 ##0.0001
epochs=100
height=768
width=512

PATH

In [13]:
dataset_path = os.path.join(drive_path, "Colab Notebooks", "dataset")

files_dir = os.path.join(drive_path, "Colab Notebooks", "files")
model_file = os.path.join(files_dir, "unet-data.h5")
log_file = os.path.join(files_dir, "log-data.csv")


CREATING FOLDER

In [8]:
def create_dir(path):
    if not os.path.exists(path):
      os.makedirs(path)

In [9]:
create_dir(files_dir)

BUILDING UNET

CONV BLOCK

In [10]:
def conv_block(inputs,num_filters):
    x=Conv2D(num_filters,3,padding="same")(inputs)
    x=BatchNormalization()(x)
    x=Activation("relu")(x)

    x=Conv2D(num_filters,3,padding="same")(x)
    x=BatchNormalization()(x)
    x=Activation("relu")(x)

    return x

ENCODER BLOCK

In [11]:
def encoder_block(inputs,num_filters):
  x=conv_block(inputs,num_filters)
  p=MaxPool2D((2,2))(x)
  return x,p

DECODER BLOCK

In [16]:
def decoder_block(inputs,skip,num_filters):
  x=Conv2DTranspose(num_filters,(2,2),strides=2,padding="same")(inputs)
  x=Concatenate()([x,skip])
  x=conv_block(x,num_filters)
  return x

UNET

In [17]:
def build_unet(input_shape):
  inputs=Input(input_shape)

  #Encoder
  s1,p1=encoder_block(inputs,64)
  s2,p2=encoder_block(p1,128)
  s3,p3=encoder_block(p2,256)
  s4,p4=encoder_block(p3,512)

  #bridge
  b1=conv_block(p4,1024)

  #decoder
  d1=decoder_block(b1,s4,512)
  d2=decoder_block(d1,s3,256)
  d3=decoder_block(d2,s2,128)
  d4=decoder_block(d3,s1,64)

  outputs=Conv2D(1,1,padding="same",activation="sigmoid")(d4)

  model=Model(inputs,outputs,name="UNET")
  return model